In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time



from IPython.display import clear_output

import random
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
BATCH_SIZE = 128
BUFFER_SIZE = 50000
os.chdir("/content/Data")
fileList = os.listdir()
os.chdir(fileList[22])
sample_train_images = np.load(fileList[22])
sample_train_images = sample_train_images.reshape(sample_train_images.shape[0], sample_train_images.shape[1], sample_train_images.shape[2],sample_train_images.shape[3]).astype('float32')
sample_train_images = ((sample_train_images -127.5) / 127.5)*1 #Esto te da valores de -1 a 1
sample_real_batch_dataset = tf.data.Dataset.from_tensor_slices(sample_train_images.reshape(sample_train_images.shape[0],sample_train_images.shape[1],sample_train_images.shape[2],sample_train_images.shape[3])).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
sample_real_image_batch = next(iter(sample_real_batch_dataset))
os.chdir("..")

# The discriminator



In [ ]:
class generate_discriminator_model(keras.Model):
  def __init__(self):
    super().__init__()

    #first model
    self.conv2D_1_16 = tf.keras.layers.Conv2D(64, kernel_size = (17,9),
                                            padding='same',  
                                            kernel_initializer='he_normal',
                                            input_shape=[16,8,4],
                                            activation = tf.nn.leaky_relu)    
    self.drop_1_16 = tf.keras.layers.Dropout(rate = .3) 
    self.conv2D_2_16 = tf.keras.layers.Conv2D(32, kernel_size = (16,8),
                                            padding='same',  
                                            kernel_initializer='he_normal',
                                            activation = tf.nn.leaky_relu)    
    self.drop_2_16 = tf.keras.layers.Dropout(rate = .1)           
    self.flatten_16 = tf.keras.layers.Flatten()
    self.dense_1_16 =tf.keras.layers.Dense(32, activation = tf.nn.leaky_relu)
    self.dense_out_16 = tf.keras.layers.Dense(1, activation = "linear")



    self.MaxPooling_32To16 = tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=None, padding="valid",
    data_format=None)    

    # Second model
    self.conv2D_1_32 = tf.keras.layers.Conv2D(16, kernel_size = (32,16),
                                            strides = (1,1),
                                            padding='same',  
                                            kernel_initializer='he_normal',
                                            input_shape=[32,16,4],
                                            activation = tf.nn.leaky_relu)
    self.drop_1_32 = tf.keras.layers.Dropout(rate = .3) 

    self.conv2D_2_32 = tf.keras.layers.Conv2D(4, kernel_size = (5,5),
                                            strides = (2,2),
                                            padding='same',  
                                            kernel_initializer='he_normal',
                                            input_shape=[32,16,4],
                                            activation = tf.nn.leaky_relu)
    self.drop_2_32 = tf.keras.layers.Dropout(rate = .1) 
        


    self.MaxPooling_64To32 = tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=None, padding="valid",
    data_format=None)    
  

  def first_call(self, input_tensor_16x8, training = True):
    x = self.conv2D_1_16(input_tensor_16x8)
    x = self.drop_1_16(x)    
    x = self.conv2D_2_16(x)
    x = self.drop_2_16(x)  
    x = self.flatten_16(x)
    x = self.dense_1_16(x)
    x = self.dense_out_16(x)

    return x  
  
  def second_call(self, input_tensor_32x16, alpha):
    x_old = self.MaxPooling_32To16(input_tensor_32x16)  #downsampling
    x_old = self.first_call(x_old)
    
    #Added layers
    x_new = self.conv2D_1_32(input_tensor_32x16)
    x_new = self.drop_1_32(x_new)
    x_new = self.conv2D_2_32(input_tensor_32x16) #converts to 16
    x_new = self.drop_2_32(x_new)    
    x_new = self.first_call(x_new)

    x = ((1-alpha) * x_old) + (alpha * x_new)

    return x


### Optimizers and loss

In [ ]:
discriminator = generate_discriminator_model()
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

In [ ]:
def discriminator_loss(discriminator_predictions_real, discriminator_predictions_fake):
  loss = -(discriminator_predictions_real) + discriminator_predictions_fake
  return tf.reduce_mean(loss)
  

# The generator

In [ ]:
class generate_generator_model(keras.Model):
  def __init__(self, latent_dim = 100):
    super().__init__()
    self.latent_dim = latent_dim

    #Model part 1
    self.dense_1_16 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu,
                                         input_shape= (self.latent_dim, ))
    
    self.dense_2_16 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
    
    self.dense_3_16 = tf.keras.layers.Dense(units=16*8*4, activation=tf.nn.leaky_relu)  

    self.reshape_1_16 = tf.keras.layers.Reshape(target_shape=(16,8,4))
    self.conv2DT_1_16 = tf.keras.layers.Conv2DTranspose(
              filters=64,
              kernel_size=(16,8),
              strides=(1, 1),
              padding="SAME",
              activation= tf.nn.leaky_relu)
    
    self.batchNorm_2_16 = tf.keras.layers.BatchNormalization()
    self.conv2DT_2_16 = tf.keras.layers.Conv2DTranspose(
              filters=16,
              kernel_size=(15,7),
              strides=(1, 1),
              padding="SAME",
              activation=tf.nn.leaky_relu)

    self.batchNorm_3_16 = tf.keras.layers.BatchNormalization()
    self.conv2DT_3_16 = tf.keras.layers.Conv2DTranspose(
              filters=16,
              kernel_size=(13,5),
              strides=(1, 1),
              padding="SAME",
              activation=tf.nn.leaky_relu)      
    
    self.batchNorm_4_16 = tf.keras.layers.BatchNormalization()
    self.conv2DT_4_16 = tf.keras.layers.Conv2DTranspose(
              filters=4,
              kernel_size=(5,5),
              strides=(1, 1),
              padding="SAME",
              activation='linear')    
    

    self.UpSampling2D_16to32 = tf.keras.layers.UpSampling2D(
        size=(2, 2), data_format=None, interpolation='nearest')
    
    # Model part 2 #################################
    self.reshape_1_32 = tf.keras.layers.Reshape(target_shape=(16, 8, 4)) #Shape at which it enters
    self.conv2DT_1_32 = tf.keras.layers.Conv2DTranspose(
              filters=8,
              kernel_size=(32,16),
              strides=(2, 2),
              padding="SAME",
              activation=tf.nn.leaky_relu)
    self.batchNorm_1_32 = tf.keras.layers.BatchNormalization()

    self.conv2DT_2_32 = tf.keras.layers.Conv2DTranspose(
              filters=16,
              kernel_size=(31,15),
              strides=(2, 2),
              padding="SAME",
              activation=tf.nn.leaky_relu)
    self.batchNorm_2_32 = tf.keras.layers.BatchNormalization()    

    self.conv2DT_3_32 = tf.keras.layers.Conv2DTranspose(
              filters=4,
              kernel_size=(5,5),
              strides=(1, 1),
              padding="SAME",
              activation='linear')
    

    self.UpSampling2D_32to64 = tf.keras.layers.UpSampling2D(
        size=(2, 2), data_format=None, interpolation='nearest')

  def call_first(self, input_noise, training = True):
      x = self.dense_1_16(input_noise)
      x = self.dense_2_16(x)  
      x = self.dense_3_16(x)  
      x = self.reshape_1_16(x)
      x = self.conv2DT_1_16(x)
      x = self.batchNorm_2_16(x)
      x = self.conv2DT_2_16(x)
      x = self.batchNorm_3_16(x)
      x = self.conv2DT_3_16(x)
      x = self.batchNorm_4_16(x)
      x = self.conv2DT_4_16(x)    
      #x = tf.clip_by_value(x,-1,1) #should I?
      return x
  
  def call_second(self, input_noise, alpha):
    x_old = self.call_first(input_noise)
    x_old = self.UpSampling2D_16to32(x_old) # now we get 32 image

    ########################
    # Now comes the model but with the new layer (32)
    x_new = self.call_first(input_noise)
    
    # Model 2
    x_new = self.reshape_1_32(x_new)
    x_new = self.conv2DT_1_32(x_new)
    x_new = self.batchNorm_1_32(x_new)
    x_new = self.conv2DT_2_32(x_new)
    x_new = self.batchNorm_2_32(x_new)
    x_new = self.conv2DT_3_32(x_new)

    x = ((1-alpha) * x_old) + (alpha * x_new)

    return x


### Generator Optimizers and loss


In [ ]:
generator = generate_generator_model()
generator_optimizer = tf.keras.optimizers.Adam(1e-5)

In [ ]:
def generator_loss(d_fake_predictions):
  loss = -tf.reduce_mean(d_fake_predictions)
  return loss

# Train the model

This needs to be "modular", when one model at one resolution converges, we do the next resolution.

I will do it manually.
(train one, then second one)

Define checkpoints

In [ ]:
pwd

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

The training functions

In [ ]:
def get_reduced_weights_or_gradients(values):
    for values_i in range(len(values)):
            if values_i == 0:
                values_red = tf.reduce_mean(values[values_i])
            else:
                values_red += tf.reduce_mean(values[values_i])

    values_red = values_red/len(values)    
    
    return values_red
    

In [ ]:
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape:
        for _ in range(3):
          noise = tf.random.normal([BATCH_SIZE, 100])
          generated_images = generator.call_first(noise)
          with tf.GradientTape() as disc_tape:
                discriminator_predictions_fake = discriminator.first_call(generated_images)
                discriminator_predictions_real = discriminator.first_call(images)
                ## Gradient penalty process
                with tf.GradientTape() as disc_tape_gp:
                    epsilon = tf.random.uniform([BATCH_SIZE,1,1,1], minval=0, maxval = 1)
                    differences = generated_images - images
                    interpolates  = images + (epsilon * differences)
                    disc_tape_gp.watch(interpolates)
                    discriminator_predictions_interpolates = discriminator.first_call(interpolates)
                    gradients_interpolates = disc_tape_gp.gradient(discriminator_predictions_interpolates,
                                                               [interpolates])[0] # Not sure why the [0] or the [inter]

                    slopes = tf.sqrt(tf.reduce_sum(tf.math.square(gradients_interpolates), axis = [1,2,3]))
                    gradient_penalty = 10 * tf.reduce_mean((slopes - 1)**2)

                    disc_loss = discriminator_loss(discriminator_predictions_real, discriminator_predictions_fake)
                    disc_loss += gradient_penalty

                gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)  
                discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        gen_loss = generator_loss(discriminator_predictions_fake)

        gradients_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables)  
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    

    disc_grads_reduced = get_reduced_weights_or_gradients(gradients_of_discriminator)
    disc_weights_reduced = get_reduced_weights_or_gradients(discriminator.trainable_variables)
    gen_grads_reduced = get_reduced_weights_or_gradients(gradients_of_generator)
    gen_weights_reduced = get_reduced_weights_or_gradients(generator.trainable_variables)
    
    
    return tf.reduce_mean(discriminator_predictions_real), tf.reduce_mean(discriminator_predictions_fake), disc_loss, disc_grads_reduced, gen_grads_reduced, disc_weights_reduced,gen_weights_reduced
  

@tf.function
def train_step_2(images,alpha):
    with tf.GradientTape() as gen_tape:
        for _ in range(3):
          noise = tf.random.normal([BATCH_SIZE, 100])
          generated_images = generator.call_second(noise, alpha)
          with tf.GradientTape() as disc_tape:
                discriminator_predictions_fake = discriminator.second_call(generated_images, alpha) #Change call
                discriminator_predictions_real = discriminator.second_call(images, alpha) #Change call
                ## Gradient penalty process
                with tf.GradientTape() as disc_tape_gp:
                    epsilon = tf.random.uniform([BATCH_SIZE,1,1,1], minval=0, maxval = 1)
                    differences = generated_images - images
                    interpolates  = images + (epsilon * differences)
                    disc_tape_gp.watch(interpolates)
                    discriminator_predictions_interpolates = discriminator.second_call(interpolates, alpha) #Change call
                    gradients_interpolates = disc_tape_gp.gradient(discriminator_predictions_interpolates,
                                                               [interpolates])[0] # Not sure why the [0] or the [inter]

                    slopes = tf.sqrt(tf.reduce_sum(tf.math.square(gradients_interpolates), axis = [1,2,3]))
                    gradient_penalty = 10 * tf.reduce_mean((slopes - 1)**2)

                    disc_loss = discriminator_loss(discriminator_predictions_real, discriminator_predictions_fake)
                    disc_loss += gradient_penalty

                gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)  
                discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))                

        gen_loss = generator_loss(discriminator_predictions_fake)

        gradients_of_generator = gen_tape.gradient(gen_loss,generator.trainable_variables)  
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    disc_grads_reduced = get_reduced_weights_or_gradients(gradients_of_discriminator)
    disc_weights_reduced = get_reduced_weights_or_gradients(discriminator.trainable_variables)
    gen_grads_reduced = get_reduced_weights_or_gradients(gradients_of_generator)
    gen_weights_reduced = get_reduced_weights_or_gradients(generator.trainable_variables)

    return tf.reduce_mean(discriminator_predictions_real), tf.reduce_mean(discriminator_predictions_fake), disc_loss, disc_grads_reduced, gen_grads_reduced, disc_weights_reduced,gen_weights_reduced
    

In [ ]:
discLosses = []
discGrads = []
genGrads = []
discWeights = []
genWeights = []

def train(dataset, epochs, dataset_num,call_number,alpha):

  for epoch in range(epochs):
    start = time.time()
    for image_batch in dataset:
      if(call_number == 1):
        disc_pred_real, disc_pred_fake, disc_loss, disc_grads,gen_grads,disc_weight,gen_weight = train_step(image_batch)    
      elif(call_number == 2):
        disc_pred_real, disc_pred_fake, disc_loss, disc_grads,gen_grads,disc_weight,gen_weight = train_step_2(image_batch,alpha = alpha)    
      
      discLosses.append(disc_loss)
      discGrads.append(disc_grads)
      genGrads.append(gen_grads)
      discWeights.append(disc_weight)
      genWeights.append(gen_weight)
      
    # Save the model every X epochs
    if (epoch + 1) % 30 == 0:
      #checkpoint.save(file_prefix = checkpoint_prefix)
      print("disc_pred_real ", disc_pred_real, "disc_pred_fake", disc_pred_fake)
      print(" disc loss: ", disc_loss)
      print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start), " dataset_num: ", dataset_num)
    
      noise = tf.random.normal([2, 100]) #Batch size changed to 2
      if(call_number == 1):
        sample_image = generator.call_first(noise)[0]
      elif(call_number == 2):
        sample_image = generator.call_second(noise, alpha = alpha)[0]
      
      fig = plt.figure(figsize = (15,3))
      plt.subplot(1,3,1)
      plt.imshow(sample_image)
      plt.subplot(1,3,2)
      plt.plot(np.array(discWeights).tolist())
      plt.subplot(1,3,3)
      plt.plot(np.array(genWeights).tolist())        
      #plt.axis('off')
      plt.show()
      
      fig2 = plt.figure(figsize = (15,3))
      plt.subplot(1,3,1)
      plt.plot(np.array(discGrads).tolist())
      plt.subplot(1,3,2)
      plt.plot(np.array(genGrads).tolist())
      plt.subplot(1,3,3)
      plt.plot(np.array(discLosses).tolist())
      plt.tight_layout()
      plt.show()
      clear_output(wait=True)
  # Generate after the final epoch
  clear_output(wait=True)

For the first loop I need to downsample the real images.


In [ ]:
maxPooling2D = tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=None, padding="valid", data_format=None
)

### Train the first model (32x16)

In [ ]:
os.chdir("/content/Data")
fileList = os.listdir()
EPOCHS = 200
try:
  fileList.remove(".ipynb_checkpoints")
  fileList.remove("training_checkpoints")
  fileList.remove("training_checkpoints.zip")
except: 
  pass
dataset_num = 0
indexList = list(range(len(fileList)))
call_number = 2
alpha = 0
for i in range(len(fileList)):
    val = random.choice(indexList)
    indexList.remove(val)
    os.chdir(fileList[val])
    train_images = np.load(fileList[val])
    train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2],train_images.shape[3]).astype('float32')
    train_images = ((train_images-127.5) / 127.5)*1 #Esto te da valores de -.1 a .1
    
    train_images = maxPooling2D(train_images) # serían 128x64
    train_images =  maxPooling2D(train_images) # serían 64x32  
    train_images =  maxPooling2D(train_images) # serían 32x16    
    #train_images =  maxPooling2D(train_images) # serían 16x8       
    train_images = np.array(train_images)    
    os.chdir("..")     
    # Porque son los que regresa el generador
    batch_dataset = tf.data.Dataset.from_tensor_slices(train_images.reshape(train_images.shape[0],train_images.shape[1],train_images.shape[2],train_images.shape[3])).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    train(batch_dataset, EPOCHS, dataset_num,call_number = call_number, alpha = alpha)
    
    if alpha < 1:
      alpha += .05
    elif alpha == 1:
      pass
    else: 
      alpha = 1

    print("dataset_num: ", dataset_num)
    dataset_num += 1

### Test the models and analyze pixel distribution

In [ ]:
noise = tf.random.normal([BATCH_SIZE, 100])
generated_images = generator.call_second(noise,1)
plt.imshow(generated_images[0])

In [ ]:
discriminator.second_call(generated_images,1)[0]

In [ ]:
plt.imshow(train_images[100])

In [ ]:
flatGenerations = (tf.reshape(generated_images, [-1]))
plt.hist(flatGenerations)

In [ ]:
flatImages = (tf.reshape(train_images, [-1]))
plt.hist(flatImages)

In [ ]:
!zip -r /content/Data/training_checkpoints /content/Data/training_checkpoints